In [ ]:
import os
import json
import pandas as pd
from utils.preprocess_utils import identify_future_failures_dict

In [ ]:
pd.set_option('display.max_columns', None)

### 0. Load Preprocessed Data

In [ ]:
DATA_DIR = './data/'
SAVE_DATA_DIR = os.path.join(DATA_DIR, 'targets_data/')
PREPROCESSED_DATA_F = 'preprocessed_data_variable_hour_window_cols.csv'
PREPROCESSED_DATA_LOC = os.path.join(DATA_DIR, PREPROCESSED_DATA_F)

In [ ]:
df = pd.read_csv(
    PREPROCESSED_DATA_LOC,
)
print(f"- {format(len(df), ',')} total rows in df.")
df.head(3)

- 876,100 total rows in df.


,datetime,machineID,volt,rotate,pressure,vibration,12_machineID_time_window,12_order_in_time_window,12_machine_example_ID,12_step_ID,24_machineID_time_window,24_order_in_time_window,24_machine_example_ID,24_step_ID,48_machineID_time_window,48_order_in_time_window,48_machine_example_ID,48_step_ID,72_machineID_time_window,72_order_in_time_window,72_machine_example_ID,72_step_ID,comp1_maint,comp2_maint,comp3_maint,comp4_maint,error1,error2,error3,error4,error5,comp1_failure,comp2_failure,comp3_failure,comp4_failure,age,model2,model3,model4
0,2015-01-01 06:00:00,1,176.217853,418.504078,113.077935,45.087686,1,1,"(001, 001)","(001, 001, 001)",1,1,"(001, 001)","(001, 001, 001)",1,1,"(001, 001)","(001, 001, 001)",1,1,"(001, 001)","(001, 001, 001)",0,0,0,0,0,0,0,0,0,0,0,0,0,18,False,True,False
1,2015-01-01 07:00:00,1,162.879223,402.747490,95.460525,43.413973,1,2,"(001, 001)","(001, 001, 002)",1,2,"(001, 001)","(001, 001, 002)",1,2,"(001, 001)","(001, 001, 002)",1,2,"(001, 001)","(001, 001, 002)",0,0,0,0,0,0,0,0,0,0,0,0,0,18,False,True,False
2,2015-01-01 08:00:00,1,170.989902,527.349825,75.237905,34.178847,1,3,"(001, 001)","(001, 001, 003)",1,3,"(001, 001)","(001, 001, 003)",1,3,"(001, 001)","(001, 001, 003)",1,3,"(001, 001)","(001, 001, 003)",0,0,0,0,0,0,0,0,0,0,0,0,0,18,False,True,False


In [ ]:
# only keeping certain columns of interest
df = df[
    [
        'datetime', 'machineID',
        '12_machineID_time_window', '12_order_in_time_window', '12_machine_example_ID', '12_step_ID',
        '24_machineID_time_window', '24_order_in_time_window', '24_machine_example_ID', '24_step_ID',
        '48_machineID_time_window', '48_order_in_time_window', '48_machine_example_ID', '48_step_ID',
        '72_machineID_time_window', '72_order_in_time_window', '72_machine_example_ID', '72_step_ID',
        'age',
        'model2', 'model3', 'model4',
        'volt', 'rotate', 'pressure', 'vibration',
        'comp1_maint', 'comp2_maint', 'comp3_maint', 'comp4_maint',
        'error1', 'error2', 'error3', 'error4', 'error5',
        'comp1_failure', 'comp2_failure', 'comp3_failure', 'comp4_failure',
    ]
]

In [ ]:
# typecasting columns
df['datetime'] = pd.to_datetime(df['datetime'])
df['machineID'] = df['machineID'].astype(int)

df['12_machineID_time_window'] = df['12_machineID_time_window'].astype(int)
df['12_order_in_time_window'] = df['12_order_in_time_window'].astype(int)
df['12_machine_example_ID'] = df['12_machine_example_ID'].astype(str)
df['12_step_ID'] = df['12_step_ID'].astype(str)
df['24_machineID_time_window'] = df['24_machineID_time_window'].astype(int)
df['24_order_in_time_window'] = df['24_order_in_time_window'].astype(int)
df['24_machine_example_ID'] = df['24_machine_example_ID'].astype(str)
df['24_step_ID'] = df['24_step_ID'].astype(str)
df['48_machineID_time_window'] = df['48_machineID_time_window'].astype(int)
df['48_order_in_time_window'] = df['48_order_in_time_window'].astype(int)
df['48_machine_example_ID'] = df['48_machine_example_ID'].astype(str)
df['48_step_ID'] = df['48_step_ID'].astype(str)
df['72_machineID_time_window'] = df['72_machineID_time_window'].astype(int)
df['72_order_in_time_window'] = df['72_order_in_time_window'].astype(int)
df['72_machine_example_ID'] = df['72_machine_example_ID'].astype(str)
df['72_step_ID'] = df['72_step_ID'].astype(str)

df['age'] = df['age'].astype(int)
df['model2'] = df['model2'].astype(int)
df['model3'] = df['model3'].astype(int)
df['model4'] = df['model4'].astype(int)

df['volt'] = df['volt'].astype(float)
df['rotate'] = df['rotate'].astype(float)
df['pressure'] = df['pressure'].astype(float)
df['vibration'] = df['vibration'].astype(float)
df['comp1_maint'] = df['comp1_maint'].astype(int)
df['comp2_maint'] = df['comp2_maint'].astype(int)
df['comp3_maint'] = df['comp3_maint'].astype(int)
df['comp4_maint'] = df['comp4_maint'].astype(int)
df['error1'] = df['error1'].astype(int)
df['error2'] = df['error2'].astype(int)
df['error3'] = df['error3'].astype(int)
df['error4'] = df['error4'].astype(int)
df['error5'] = df['error5'].astype(int)
df['comp1_failure'] = df['comp1_failure'].astype(int)
df['comp2_failure'] = df['comp2_failure'].astype(int)
df['comp3_failure'] = df['comp3_failure'].astype(int)
df['comp4_failure'] = df['comp4_failure'].astype(int)

In [ ]:
df.head(3)

,datetime,machineID,12_machineID_time_window,12_order_in_time_window,12_machine_example_ID,12_step_ID,24_machineID_time_window,24_order_in_time_window,24_machine_example_ID,24_step_ID,48_machineID_time_window,48_order_in_time_window,48_machine_example_ID,48_step_ID,72_machineID_time_window,72_order_in_time_window,72_machine_example_ID,72_step_ID,age,model2,model3,model4,volt,rotate,pressure,vibration,comp1_maint,comp2_maint,comp3_maint,comp4_maint,error1,error2,error3,error4,error5,comp1_failure,comp2_failure,comp3_failure,comp4_failure
0,2015-01-01 06:00:00,1,1,1,"(001, 001)","(001, 001, 001)",1,1,"(001, 001)","(001, 001, 001)",1,1,"(001, 001)","(001, 001, 001)",1,1,"(001, 001)","(001, 001, 001)",18,0,1,0,176.217853,418.504078,113.077935,45.087686,0,0,0,0,0,0,0,0,0,0,0,0,0
1,2015-01-01 07:00:00,1,1,2,"(001, 001)","(001, 001, 002)",1,2,"(001, 001)","(001, 001, 002)",1,2,"(001, 001)","(001, 001, 002)",1,2,"(001, 001)","(001, 001, 002)",18,0,1,0,162.879223,402.747490,95.460525,43.413973,0,0,0,0,0,0,0,0,0,0,0,0,0
2,2015-01-01 08:00:00,1,1,3,"(001, 001)","(001, 001, 003)",1,3,"(001, 001)","(001, 001, 003)",1,3,"(001, 001)","(001, 001, 003)",1,3,"(001, 001)","(001, 001, 003)",18,0,1,0,170.989902,527.349825,75.237905,34.178847,0,0,0,0,0,0,0,0,0,0,0,0,0


### 1. Create Targets Data Dicts From Preprocessed Data

In [ ]:
for hour_feature_window in [12, 24, 48, 72]:
    print(f"- Hour Feature Window: {hour_feature_window}")

    for hour_lookahead in [1, 6, 24, 72, 168]:

        # # skip if already done
        # if hour_feature_window == 12:
        #     if hour_lookahead in [1, 6, 24, 168]:
        #         print(f"*Skipping (hour_feature_window, hour_lookahead) = ({hour_feature_window}, {hour_lookahead})")
        #         continue

        print(f"\t- Hours Lookahead: {hour_lookahead}")

        targets_dict = identify_future_failures_dict(
            df = df,
            machine_example_id_col = f'{hour_feature_window}_machine_example_ID',
            hours_ahead = hour_lookahead
        )
        print("\n")

        prevalence = sum(v == 1 for v in targets_dict.values()) / len(targets_dict)
        print(f"\t\t- {format(len(targets_dict), ',')} total key-value pairs in targets_dict.")
        print(f"\t\t\t- Prevalence: {round(100 * prevalence, 2)}%.")

        # remove max order in time window key (there is no subsequent outcome on which to judge its outcome)
        max_order_in_time_window = df[f'{hour_feature_window}_order_in_time_window'].max()
        targets_dict = {
            k: v for k, v in targets_dict.items()
            if int(k[6:9]) != max_order_in_time_window
        }
        prevalence = sum(v == 1 for v in targets_dict.values()) / len(targets_dict)
        print(f"\t\t- {format(len(targets_dict), ',')} total key-value pairs in targets_dict.")
        print(f"\t\t\t- Prevalence: {round(100 * prevalence, 2)}%.")

        # save as JSON
        SAVE_FILENAME = f'targets_dict_anyfailure_{hour_feature_window}hrfeaturewindow_{hour_lookahead}hrlookahead.json'
        SAVE_LOC = os.path.join(SAVE_DATA_DIR, SAVE_FILENAME)
        with open(SAVE_LOC, mode = 'w') as json_f:
          json.dump(obj = targets_dict, fp = json_f, indent = 4)
        print(f"\t\t*Saved file: {SAVE_FILENAME}")

- Hour Feature Window: 12
*Skipping (hour_feature_window, hour_lookahead) = (12, 1)
*Skipping (hour_feature_window, hour_lookahead) = (12, 6)
*Skipping (hour_feature_window, hour_lookahead) = (12, 24)
	- Hours Lookahead: 72
1   2   3   4   5   6   7   8   9   10   11   12   13   14   15   16   17   18   19   20   21   22   23   24   25   26   27   28   29   30   31   32   33   34   35   36   37   38   39   40   41   42   43   44   45   46   47   48   49   50   51   52   53   54   55   56   57   58   59   60   61   62   63   64   65   66   67   68   69   70   71   72   73   74   75   76   77   78   79   80   81   82   83   84   85   86   87   88   89   90   91   92   93   94   95   96   97   98   99   100   

		- 73,100 total key-value pairs in targets_dict.
			- Prevalence: 5.77%.
		- 73,000 total key-value pairs in targets_dict.
			- Prevalence: 5.77%.
		*Saved file: targets_dict_anyfailure_12hrfeaturewindow_72hrlookahead.json
*Skipping (hour_feature_window, hour_lookahead) = (12, 168